# I. DATA

## 1. Download and examine our data
Let's see what's in our data folder.

In [ ]:
# Use the "!ls" command to list your directory contents. 
# The preceding exclamation mark tells Jupyter to run a bash command instead of Python


In [ ]:
# import the Image class from PIL, the Python Image Library


# open an image file of choice



## 2. Instantiate Dataset object

**Purpose:** index and load the image files we want to use for our dataset, assign numeric labels to each class.

We can either define a custom _Dataset_ class, or import a prebuilt _Dataset_ from the `torchvision` package. Because this is our first tutorial, let's use a prebuilt one.

According to the <a href="https://pytorch.org/docs/stable/torchvision/datasets.html#imagefolder">docs</a>, an ImageFolder Dataset helps load data arranged like this:
```
root/dog/xxx.png
root/dog/xxy.png
root/dog/xxz.png

root/cat/123.png
root/cat/nsdf3.png
root/cat/asd932_.png
```

Here is the class signature. We at least need a **root directory**, but we should also pass in a **transform**.

`torchvision.datasets.ImageFolder(root, transform=None, target_transform=None, loader=<function default_loader>, is_valid_file=None)`

In [ ]:
# import ImageFolder from torchvision.datasets


Our transform will do the following:
<img src="media/transform.jpg" width=600/>

In [ ]:
# import transforms from torchvision

# Declare transforms to (1) resize images (2) turn PIL Images into PyTorch Tensors




In [ ]:
# instantiate an ImageFolder object



In [ ]:
# The Dataset object is indexable, just like an array. Try accessing the 0th index, what gets returned?


**Problem:** this basic iterator only loads one image at a time. But the power of a GPU comes from running multiple calculations in _parallel_. How can we load whole batches of images at once?

## 3. Declare DataLoader and data transforms

**Purpose:** load data in the desired batch size, with shuffling, using multiple CPU threads for efficiency.

In [ ]:
# import the DataLoader class from torch.utils.data


# instantiate a DataLoader object, pass in our instantiated Dataset object. Try examining its attributes



In [ ]:
# what's in our dataloader?


# check the shape of our data! notice how it prepends an extra batch dimension



# II. MODEL (Neural Network)

**Purpose:** define our neural network architecture.

We can either define a custom architecture that extends _nn.Module_, or import a predefined architecture from `torchvision`. Because this is our first tutorial, let's again use torchvision.

Here's a list of officially available models: https://pytorch.org/docs/stable/torchvision/models.html

(Even more unofficial implementations are available <a href="https://github.com/Cadene/pretrained-models.pytorch">here</a>)

In [ ]:
# import torchvision.models

# fetch an instance of AlexNet

# examine the model parameters


Diagram of AlexNet:
<img src="media/alexnet.png" width=500/>

The output layer represents the number of classes. The index with the highest activation is the predicted class.

In [ ]:
# quick check, pass an image through our network


# what's the shape of the output?


The output of this default architecture is for 1000 classes, not 2 classes for our problem. Let's add a final layer at the end. Notice how the architecture parameters ends with "classifier?" We'll modify that.

In [ ]:
# import torch.nn

# overwrite the "classifier" to add a Linear 1000->2 layer at the end



In [ ]:
# quick test, pass an image through our network. output should now be size 2



# III. LOSS 
(a.k.a. cost or error function)

**Purpose:** calculate how wrong the neural network's predictions are.

Why do we need to choose a loss? Because there are multiple ways to define what error means, e.g. L1 error, L2 error. Let's use [CrossEntropyLoss](https://pytorch.org/docs/stable/nn.html#crossentropyloss).

In [ ]:
# instantiate CrossEntropyLoss object


In [ ]:
# run a quick test


# IV. OPTIMIZER

**Purpose:** decide how the model should respond to error.

See official available optimizers [here](https://pytorch.org/docs/stable/optim.html).

Examples of different optimization strategies on varying error surfaces.

<table>
<tr>
    <td> <img src="media/optimizer_1.gif" height="275" /> </td>
    <td> <img src="media/optimizer_2.gif" height="275" /> </td>
    <td> <img src="media/optimizer_3.gif" height="275" /> </td>
</tr>
</table>

In [ ]:
# import torch.optim

# instantiate an optimizer


# V. TRAIN LOOP

In [ ]:
# create a device object for GPU usage


# move our model to the device


In [1]:
# train for n epochs. an epoch is a full iteration through our dataset


# create something to track of accuracy over time


# loop over epochs

    
    # track our accuracy

    
    # loop over our data loader


        
        # pass data through model

        # calculate the loss

        
        # Use our optimizer to update the network
        # 1: zero_grad our optimizer

        # 2: run a backward pass

        # 3: make a step

        
        # calculate predictions so we can track accuracy
        
        
    
    # calculate the accuracy by dividing correct by length of the dataset
    
    
    # append accuracy to our list
    

In [ ]:
# make a simple line plot using pandas, ylim between 0 and 1



# VI. VISUALIZE RESULTS

**Consider:** How accurate was your model? How confident were its predictions? Does it make clear-cut decisions?

Feel free to use the below function to visualize results. I won't go through it because its details is not part of this tutorial.

In [ ]:
import matplotlib.pyplot as plt  # graphical library, to plot images
# special Jupyter notebook command to show plots inline instead of in a new window
%matplotlib inline

import os
from glob import glob
from math import floor

def visualize_results(data_root, model, transforms):
    set_eval = False
    if model.training:
        set_eval=True
        model.eval()
    
    val_dataset = ImageFolder(data_root, transforms)
    val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=len(val_dataset))
    data, labels = next(iter(val_loader))
    data, labels = data.to(device), labels.to(device)
    outputs = my_net(data)
    _, preds = torch.max(outputs, dim=1)
    num_correct = torch.sum(preds==labels.data).item()
    pred_probs = torch.nn.functional.softmax(outputs, dim=-1).cpu().data.numpy()
    

    print("VALIDATION ACCURACY:", num_correct / len(val_dataset))

    # show the probabilities for each picture
    fig, axs = plt.subplots(6, 5, figsize=(20, 20))
    images = [Image.open(img_path) for img_path in list(zip(*val_dataset.samples))[0]]
    for i, img in enumerate(images):
        ax = axs[floor(i/5)][i % 5]
        ax.axis('off')
        ax.set_title("{:.0f}% Chi, {:.0f}% Muff".format(100*pred_probs[i,0], 100*pred_probs[i,1]), fontsize=18)
        ax.imshow(img)
    
    if set_eval:
        model.train()

In [2]:
# call the function visualize_results()


## Congratulations! You've successfully trained a neural network!

# Can You Do Better?

Now that we've shown you how to train a neural network, can you improve the validation accuracy by tweaking the parameters? **We challenge you to reach 100% accuracy!** (hint, it's not too hard).

Some parameters to play with:
- Number of epochs
- The model type
- The learning rate "lr" parameter in the optimizer
- The type of optimizer (https://pytorch.org/docs/stable/optim.html)
- Number of layers and layer dimensions
- Image size
- Data augmentation transforms (https://pytorch.org/docs/stable/torchvision/transforms.html)